In [1]:
# predefinitions
import pandas as pd
directory = "altman z_all(sheet1-2).xlsx"
infoFile = "name_sector_country_acs.xlsx"

In [2]:
def getCompanyHistory(filename, skipRows):
    """
    Generates a Data Frame from an excel table of altman z values of companies throughout the years also runs a simple
    NAN value and duplicate removal checks
    
    :param filename: an Excel file (.xlsx) containing a table of companies and
                    their Altman z values over the years
    :type filename: str
    
    :param skipRows: a list of indexes, indicating which rows to skip from the
                    top of the file
    :type skipRows: list of int >= 0

    :returns: A Data Frame containing Altman Z values of companies
    :rtype: pandas.DataFrame
    
    Example:
    >> getCompanyHistory("altman z_all(sheet1-2).xlsx","Sheet1", [0,1,2,4,5],
                             ["#N/A N/A", "#N/A Review"])
    """
    # extracting data
    cols = pd.read_excel(filename, encoding="latin-1", skiprows = skipRows,
                         na_values = ["#N/A N/A", "#N/A Review"], sheet_name = "Sheet1",
                         header=None, nrows=1, index_col = 0).values[0]
    data = pd.read_excel(filename, encoding="latin-1",
                           skiprows = skipRows, na_values = ["#N/A N/A", "#N/A Review"],
                           sheet_name = "Sheet1", index_col = 0)
    
    # modificatons on the extracted data
    data.columns = cols # rename columns
    data = data.dropna(axis=1,how="all") # drop columns with no values
    data = data.loc[:,~data.columns.duplicated()] # drop duplicated columns
    return data 

In [3]:
def getCompanyInfo(filename, skipRows):
    """
    :param filename: an Excel file (.xlsx) containing a table of companies and
                    their information like full name, sector, country and accounting standard
    :type filename: str
    :param skipRows: a list of indexes, indicating which rows to skip from the
                    top of the file
    :type skipRows: list of int >= 0
    :returns: A Data Frame containing information of companies
    :rtype: pandas.DataFrame

    Example:
    >> getCompanyInfo("name_sector_country_acs.xlsx",[0,1,2,3,4,5,6,7])
    """
    
    df = pd.read_excel(filename, encoding="latin-1",skiprows = skipRows, 
                           sheet_name = "Sheet1", index_col = 0)
    return df

In [4]:
def initialFilter(dataFrame, altman_z_threshold = 1.8):
    """
    This initial filter, filters companies (as default) for the highest Altman Z value of 1.8. After this filter got applied 
    to the 6k companies extracted from Bloomberg terminal another extraction operation acquired the second data set 
    specifically for these filtered companies, the info file. The info file has detailed information about companies that 
    allowed further filtering based on their GICS industry group names.
    
    Thus this function is further called with varried Altman Z thresholds.
    
    :param dataFrame: A Data Frame containing Altman Z values of companies
    :type dataFrame: pandas.DataFrame
    
    :param altman_z_threshold: A floating point number indicating the Altman Z threshold. By default it is 1.8.
    :type float: pandas.float64

    :returns: A Data Frame containing only the Altman Z values of companies that have dropped below the given threshold
    :rtype: pandas.DataFrame
    """
    
    data_lower = dataFrame.where(dataFrame <= altman_z_threshold).dropna(axis=1,how="all")
    columns_to_keep = list(data_lower.columns)
    dataFrame = dataFrame.filter(columns_to_keep,axis=1)
    
    return dataFrame

In [5]:
def dataFilter(dataFrame, mask = False):

    """
    This data filter returns the given data frame masked or unmasked, aggregating according to respective Altman Z score
    thresholds relying on GICS Industry Group of the company derived from the given data frame
    
    For mask False, and as default returns the joined data frame of Altman Z scores that contain only the companies that have 
    dropped below their own industry specific Altman Z threshold at least once
    
    Otherwise returns the joined and masked data frame of Altman Z scores that only contain companies that have dropped below 
    their own industry specific Altman Z threshold at least once
    
    :param dataFrame: A Data Frame containing Altman Z values of companies and other company information most 
    importantly GICS Industry group
    :type dataFrame: pandas.DataFrame
    
    :param mask: A boolean indicating the desired aggregation result. Default is False, unmasked, the joined 
    data frame of Altman Z scores that contain only the companies that have dropped below their own industry 
    specific Altman Z threshold at least once
    :type bool: pandas.bool

    :returns: An aggregated Data Frame according to the parameter mask
    :rtype: pandas.DataFrame
    """

    #GICS Industry groups that have different Altman Z thresholds
    ind1 = ["Materials", "Capital Goods", "Commercial & Professional Serv", "Transportation"]
    ind2 = ["Consumer Services"]

    #Respective Altman Z threshold values
    ind1_threshhold = 1.23
    ind2_threshhold = 1.1

    #Respective Data Frames containing only the companies that belong to that industry group
    data_ind1 = dataFrame.T[dataFrame.T["GICS_INDUSTRY_GROUP_NAME"].isin(ind1)].T
    data_ind2 = dataFrame.T[dataFrame.T["GICS_INDUSTRY_GROUP_NAME"].isin(ind2)].T

    #A joined data frame of the unfiltered companies of the both industry groups
    data_industries_joined = data_ind1.join(data_ind2)
    
    
    #LOWERS    
    #Respective Data Frames that are filtered to only contain companies that have dropped at least once  
    #below their respective Altman Z treshold
    data_ind1_lower = initialFilter(data_ind1[:82], ind1_threshhold)
    data_ind2_lower = initialFilter(data_ind2[:82], ind2_threshhold)
    
    #A joined data frame of the filtered companies of the both industry groups
    data_lowers_joined = data_ind1_lower.join(data_ind2_lower)
    
    
    #MASKS
    #Repective Data Frames that after being filtered are masked, to contain only value of 1 or 0 according to whether they 
    #are above or below theier given Altman Z thresholds repectively
    data_ind1_lower_masked = data_ind1_lower.notna().applymap(lambda x: 1 if x else 0)
    data_ind2_lower_masked = data_ind2_lower.notna().applymap(lambda x: 1 if x else 0)
    
    #A masked then joined data frame of the filtered companies of the both industry groups
    data_lowers_masked_joined = data_ind1_lower_masked.join(data_ind2_lower_masked)
    
    #REDUCED
    #A reduced Data Frame that doesn't include companies that belong to ind1 and ind2 groups
    columns_to_drop=list(data_industries_joined.columns)
    dataFrame_reduced = dataFrame.drop(columns_to_drop,axis=1)
    
    #Masked data frame of the reduced data frame 
    dataFrame_reduced_masked = dataFrame_reduced.where(dataFrame_reduced[:82] <= 1.8).notna().applymap(lambda x: 1 if x else 0) 
    
    #REBUILD
    #A joined data frame of Altman Z scores that only contains companies that have dropped below 
    #their own industry specific Altman Z threshold at least once
    data_rebuilt = dataFrame_reduced.join(data_lowers_joined)[:82]
    
    #Same joined data frame but instead of Altman Z score values it is masked to contain 1 or 0 values
    data_rebuilt_masked = dataFrame_reduced_masked.join(data_lowers_masked_joined)[:82]
    
    if(mask == False): 
        return data_rebuilt
    else:
        return data_rebuilt_masked


In [6]:
def loops(dictionary_of_arrays_groupby):
    
    #A helper function that counts the same numbers in a row and puts them in a list with the length of the sequences
        
    list_of_times = []
    for item in dictionary_of_arrays_groupby.items():
        list_of_times.append(item[-1])
    final_list = []
    for item in list_of_times:
        final_list.append(item[-1]+1)
    
    return final_list

In [7]:
#Reading the data
data_read = getCompanyHistory(directory, [0,1,2,4,5])
data_info = getCompanyInfo(infoFile, [0,1,2,3,4,5,6,7])

#The initial filter
data_initial = initialFilter(data_read)

#The joined data frame containing only the necessary bits for further data aggregation 
#(Line that contains GICS Industry Groups Information)
data_joined = pd.concat((data_initial,data_info[2:3]), join='inner')

In [8]:
#Modifications to data to extract relevant information 
#Separate count of each consecutive period that a company spent below their respective Altman Z score threshold
data_masked_counted = dataFilter(data_joined, True).apply(lambda y: y * (y.groupby((y != y.shift()).cumsum()).cumcount() + 1))

#Indexes the masked and counted Data Frame thus displaying each time a value repeats 
#First row is removed since it holds the amount of zeros that appear which is not utilized in further aggregations
data_mc_indexed = data_masked_counted.apply(lambda x: x.value_counts())[1:]

In [9]:
#Relevant informations such as
#The longest each company has stayed below their respective Altman Z score threshold
longest_under = data_masked_counted.apply(lambda x: x.max()).to_frame()
display(longest_under.sort_values(0, ascending = False))

#How many times each company dropped below their respective Altman Z score threshold
times_went_under = data_mc_indexed[0:1]
display(times_went_under.T)

#How long each company has stayed below their respective Altman Z score threshold each time they dropped below it
data_time_spent_under = data_mc_indexed.apply(lambda x: loops(data_mc_indexed.groupby(x).indices)).to_frame()
display(data_time_spent_under)

,0
STRL UW Equity,82.0
SKYW UW Equity,82.0
TGI UN Equity,82.0
UHAL UW Equity,82.0
AMSC UW Equity,82.0
...,...
3392 JT Equity,1.0
3349 JT Equity,1.0
3321 JT Equity,1.0
3153 JT Equity,1.0


,1.0
CVX UN Equity,1.0
VZ UN Equity,7.0
BAYN GY Equity,1.0
BMW GY Equity,1.0
CON GY Equity,2.0
...,...
WING UW Equity,1.0
ADH SJ Equity,1.0
SUI SJ Equity,1.0
TGO SJ Equity,1.0


,0
CVX UN Equity,[1]
VZ UN Equity,"[25, 6, 2, 1]"
BAYN GY Equity,[1]
BMW GY Equity,[63]
CON GY Equity,"[6, 1]"
...,...
WING UW Equity,[21]
ADH SJ Equity,[77]
SUI SJ Equity,[77]
TGO SJ Equity,[4]


In [10]:
#Join functions that merge company history, company informations and aggregated informations in a single data frame
data_merged = pd.concat((pd.concat((dataFilter(data_joined),data_info), join='inner'), longest_under.T, times_went_under, data_time_spent_under.T), join='inner')

#Renamed several indexes for clarity
data_merged_rename_temp = data_merged.rename({0: "Longest_Under", 1: "Times_Under"}, axis='index')[-1:].rename({"Longest_Under":"Each_Time_Spent_Under"}, axis='index')
data_merged_renamed = pd.concat((data_merged.rename({0: "Longest_Under", 1: "Times_Under"}, axis='index')[:90], data_merged_rename_temp), join='inner')
data_final = data_merged_renamed

display(data_final)

#Transform the final version of data frame to an excel document (commented to not recreate the excel with each execution)

#data_final.to_excel("bloomberg_all_info.xlsx")

,CVX UN Equity,VZ UN Equity,BAYN GY Equity,BMW GY Equity,CON GY Equity,DAI GY Equity,DB1 GY Equity,DTE GY Equity,DWNI GY Equity,EOAN GY Equity,...,SEAS UN Equity,TACO UR Equity,TAST UW Equity,UTI UN Equity,VVNT UN Equity,WING UW Equity,ADH SJ Equity,SUI SJ Equity,TGO SJ Equity,TSG SJ Equity
2000-03-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-09-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-12-29 00:00:00,3.4719,1.6739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-03-30 00:00:00,5.0354,1.5789,3.0633,NaN,2.3551,NaN,NaN,NaN,NaN,0.5893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACCOUNTING_STANDARD,US GAAP,US GAAP,IAS/IFRS,IAS/IFRS,IAS/IFRS,IAS/IFRS,IAS/IFRS,IAS/IFRS,IAS/IFRS,IAS/IFRS,...,US GAAP,US GAAP,US GAAP,US GAAP,US GAAP,US GAAP,IAS/IFRS,IAS/IFRS,IAS/IFRS,IAS/IFRS
COUNTRY_FULL_NAME,UNITED STATES,UNITED STATES,GERMANY,GERMANY,GERMANY,GERMANY,GERMANY,GERMANY,GERMANY,GERMANY,...,UNITED STATES,UNITED STATES,UNITED STATES,UNITED STATES,UNITED STATES,UNITED STATES,SOUTH AFRICA,SOUTH AFRICA,SOUTH AFRICA,SOUTH AFRICA
Longest_Under,1,25,1,63,6,68,72,75,48,27,...,29,26,55,67,11,21,77,77,4,75
Times_Under,1,7,1,1,2,1,1,1,1,3,...,1,1,1,1,1,1,1,1,1,1
